In [ ]:
import pandas as pd
import numpy as np

import acquire
import env

#### <span style="color:red">In writing the function to bring in my data from MySQL, I imported double the number of data points the rest of my team had imported.  Problem solved when dropping the 'USING' command in table joining query.</span>

Some Markdown text with <span style="color:blue">some *blue* text</span>.

### Importing the telco_churn data, 'customers' table, first round.  Just want to get an idea of what columns we're looking at - Round 1

**Different rounds are outlined later, as investigation should lead to the features we'll be needing most for our mvp.**


#### First Hypotheses for customer table:

#### 1.) 

- $H_0$ = contract type has no effect on churn rate
- $H_a$ = contract type DOES have an effect on churn rate

#### 2.) 

- $H_0$ = month-to-month tenure does NOT have an effect on churn rate
- $H_a$ = month-to-month tenure DOES have an effect on churn rate

#### 3.) 

- $H_0$ = contract length of one-year does NOT have an effect on churn rate
- $H_a$ = contract length of one-year DOES have an effect on churn rate

#### 4.)

- $H_0$ = contract length of two years does NOT affect churn rate
- $H_0$ = contract length of two years DOES affect churn rate

In [ ]:
telco = acquire.get_telco_data()
telco.head(10)

In [ ]:
telco.info()

#### No nulls in any of the chosen features.  Sweet.  Lots of encoding, though, because lots of un-integer objects in there.  Namely total_charges is an object when it should probably be a float.  Also, not sure (yet) if all values are unique.

**Also, just from this first pull, the categoricals are:**

- customer_id 
- phone_service ('Yes / No')
- mulitple_lines ('Yes / No')
- internet_service_type_id (1, 2, or 3)
- streaming_tv ('Yes / No') 
- streaming_movies('Yes' / No') 
- paperless_billing ('Yes / No') 
- payment_type_id
- and churn ('Yes / No')

**Leaving the continuous / numerical columns:**

- tenure (months)
- monthly_charges (monetary)
- total_charges (monetary)

In [ ]:
telco.shape

In [ ]:
telco.describe()

#### In looking at the original dataset, we discussed how phone (and other) companies cross-sell you on product offerings.  While that does increase the convenience of paying one's bills, it also anchors the customer to the company itself.  For instance, if you have a checking account at a bank, don't be surprised if they start selling you on mortgages and car loans.  

#### With this in mind, we came to the conclusion that if the customers in the dataset had multiple products with the telecomm company, there should be some weight placed on the coefficients of those variables.  Thus, a point system.

In [ ]:
# A couple of list comprehensions to put 'Yes' and 'No' values on point system.


phone_service = [0 if i == "No" else 1 for i in telco.phone_service]
mulitple_lines = [1 if i == "Yes" else 0 for i in telco.multiple_lines]

telco["phone_and_multiline"] = [phone_service[i] + mulitple_lines[i] for i in range(len(phone_service))]

#### Below we are implementing a for loop to weigh the family-type of the customers themselves.  The thought behind this being that single parents would be more likely to churn because of financial difficulties - they'd always be on the hunt for better deals.

In [ ]:
partner_and_dependents = [] # Married / Cohabitating with kids

for i in range(len(telco.partner)):
    if telco.partner[i] == "No" and telco.dependents[i] == "No":
        partner_and_dependents.append(0)
    elif telco.partner[i] == "Yes" and telco.dependents[i] == "No":
        partner_and_dependents.append(1)
    elif telco.partner[i] == "No" and telco.dependents[i] == "Yes":
        partner_and_dependents.append(2)
    elif telco.partner[i] == "Yes" and telco.dependents[i] == "Yes":
        partner_and_dependents.append(3)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
telco.shape

#### Sunny raised the question: how do we read 'No Internet Service' values.  If they're nulls, can we drop them?  The answer is 'No' because one, they make up too large a percentage of the overall data (nearly 22%) to just drop without any affect on model performance.  Second, upon closer investigation, the phrase 'No internet service' seems to be another way of stating the 'internet_service_type_id' of '3,' which does, in fact, have an influence in who / why people churn.

In [ ]:
# Running the cells below for all the columns containing 'No internet service'

telco.online_security.value_counts("No internet service")

In [ ]:
pd.crosstab(telco.online_backup, telco.online_security)

### Notes in creating the 'prepare.py' file:

<span style="color:red">*Gotta give credit where credit is due: while roles of the functions below were discussed as a team, the actual writing and walkthrough of the functions was done by Chase Thompson.  This has proven a weak area for me in the past, and through Chase's diligence and patience, my learning in how to do this has increased by leaps and bounds.* text</span>

- 'fill_na' function - though the data contained no nulls, there were, indeed, spaces as values.  This function replaces those spaces with np.nan.

- 'dropna' function - after replacing the spaces with 'NaN's (and seeing there were not enough NaN values to have an effect on the performance of our model), this function removes NaNs from the rows where they occur.

- 'phone_lines' function - list comprehension function describing the above section of the notebook outlining why a point system with regards to phone line number was important.

- 'partner_and_dependents' function - for loop function described above assigning weights to family life of individual customers.

- 'drop_columns' function - function to drop the dataframe columns we used to aggregate the values determined in the 'phone_lines' and 'partner_and_dependents' functions.

- 'X_label_encode' function - since ML algorithms can't handle strings or objects as inputs, these columns were label encoded to give them a numeric representation.  Being that LabelEncoder actually assigns a magnitude to the variables, the columns chosen to LabelEncode are considered to be important in determining customer churn rates.

- 'one_hot_encoder' function - Like LableEncoder, OneHot assigns numbers to object variables.  However, the features chosen in this function were deemed 'not as important' to customer churn rates, and OneHot minimizes the impact their magnitude would have over our overall model performance.

- 'drop_service_type' function - another aggregate function to drop the columns used in determining the 'one_hot_encoder' function.

- 'split_telco' function - function that splits telco dataset into train, validate, and test dataframes.


## Data Exploration Phase

- Import the necessary Libraries

In [ ]:
is_senior = telco.groupby("telco")["senior_citizen"].count()